# This notebook stores useful code blocks for ZND Remote Control

In [1]:
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep

In [2]:
# connect to the instrument
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()


Hello, I am: 'Rohde-Schwarz,ZND-2Port,1328517092100889,2.80'
RsInstrument driver version: 1.9.0.52
Visa manufacturer: Rohde & Schwarz GmbH
Instrument full name: ZND-2Port
Instrument installed options: K1,K6


In [2]:
def index_sequence(start_index):
    index = start_index
    while True:
        yield index
        index += 1

In [3]:
def measure():
    instrument.sweep_initiate_all()
    print("measured")

In [4]:
def memorize():
    mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
    instrument.trace_copy('MainTrace', mem_trace_name)
    instrument.trace_assign_to_window(1, next(trc_index_gen), mem_trace_name)

In [5]:
def export():
    save_path = r'C:\Users\Instrument\Desktop\Data\Measurement_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H:%M:%S")))
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)

In [7]:
def clear():
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    instrument.trace_assign_to_window(1, 1, 'MainTrace')
    mem_index_gen = index_sequence(1)
    trc_index_gen = index_sequence(2)

In [14]:
# set up the instrument
instrument.reset()
sleep(1)
instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
instrument.trace_rename(1, 'MainTrace')
# TODO: set trace points to 1000
# TODO: set timeouts
# TODO: set calibration data
# TODO: set display mode
mem_index_gen = index_sequence(1)
trc_index_gen = index_sequence(2)

In [21]:
# define buttons for manual control
instrument.button_define(Types.ButtonNumber.ONE, "Measure", measure)
instrument.button_define(Types.ButtonNumber.TWO, "Memorize", memorize)
instrument.button_define(Types.ButtonNumber.THREE, "Export", export)
instrument.button_define(Types.ButtonNumber.FOUR, "Clear", clear)
instrument.button_start_listening()